In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
###############################################
import nltk as nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import TweetTokenizer
from nltk.corpus import wordnet as wn
from nltk.corpus.reader import sentiwordnet as swn
from nltk.tag import map_tag
import re
from nltk.corpus import wordnet as wn
import codecs
import string
#################################################

import warnings
warnings.filterwarnings("error")

<h1><center> Groupe Amis d'enfance </center></h1>

<h1><center> A draft Data use cas  </center></h1>

In [2]:
ami = pd.read_csv("amis.txt",on_bad_lines='skip', header = None, sep = "-")
ami.columns = ["date","Messages"]
ami.head()

,date,Messages
0,20/07/2022 à 15:40,Les messages et les appels sont chiffrés de b...
1,19/07/2022 à 19:32,"‎Nicolas Faye a créé le groupe ""AMIS D’ENFANCE"""
2,20/07/2022 à 15:40,‎Cedriques Bampoky vous a ajouté(e)
3,20/07/2022 à 15:42,"MOI: Bonjour,"
4,Je n'ai pas vu Macky 🤣🤣🤣,NaN


In [3]:
ami.shape

(409, 2)

In [4]:
ami.isnull().sum()

date         0
Messages    43
dtype: int64

In [5]:
idx = ami.loc[pd.isna(ami["Messages"]), :].index

In [6]:
#ami.loc[idx]["Messages"] = ami.loc[idx]["date"]
ami["Messages"] = ami["Messages"].fillna(ami.loc[idx]["date"])
ami = ami.drop([0,1,2])
ami.head()

,date,Messages
3,20/07/2022 à 15:42,"MOI: Bonjour,"
4,Je n'ai pas vu Macky 🤣🤣🤣,Je n'ai pas vu Macky 🤣🤣🤣
5,20/07/2022 à 15:42,MOI: Il est finalement parti avant 2024 ? 🥲
6,20/07/2022 à 15:43,MOI: Merci @33650305387
7,20/07/2022 à 15:43,Cedriques Bampoky: Rebienvenue parmi tes frères


In [7]:
#ami = ami.dropna(axis = 0)

In [8]:
ami["date"] =  ami["date"].apply(lambda x: x.replace("à", ""))
ami.head()

,date,Messages
3,20/07/2022 15:42,"MOI: Bonjour,"
4,Je n'ai pas vu Macky 🤣🤣🤣,Je n'ai pas vu Macky 🤣🤣🤣
5,20/07/2022 15:42,MOI: Il est finalement parti avant 2024 ? 🥲
6,20/07/2022 15:43,MOI: Merci @33650305387
7,20/07/2022 15:43,Cedriques Bampoky: Rebienvenue parmi tes frères


In [9]:
for i in idx:
    ami.loc[i]["date"] = ami.loc[i-1]["date"]

In [10]:
ami.isnull().sum()

date        0
Messages    0
dtype: int64

In [11]:
ami["auteur"] = ami["Messages"].apply(lambda x : x.split(":")[0])
ami.head()

,date,Messages,auteur
3,20/07/2022 15:42,"MOI: Bonjour,",MOI
4,20/07/2022 15:42,Je n'ai pas vu Macky 🤣🤣🤣,Je n'ai pas vu Macky 🤣🤣🤣
5,20/07/2022 15:42,MOI: Il est finalement parti avant 2024 ? 🥲,MOI
6,20/07/2022 15:43,MOI: Merci @33650305387,MOI
7,20/07/2022 15:43,Cedriques Bampoky: Rebienvenue parmi tes frères,Cedriques Bampoky


In [12]:
ami["SMS"]=ami["Messages"].apply(lambda x : x.split(":")[1] if len(x.split(":"))>=2 else "Waly")

In [13]:
ami.head()

,date,Messages,auteur,SMS
3,20/07/2022 15:42,"MOI: Bonjour,",MOI,"Bonjour,"
4,20/07/2022 15:42,Je n'ai pas vu Macky 🤣🤣🤣,Je n'ai pas vu Macky 🤣🤣🤣,Waly
5,20/07/2022 15:42,MOI: Il est finalement parti avant 2024 ? 🥲,MOI,Il est finalement parti avant 2024 ? 🥲
6,20/07/2022 15:43,MOI: Merci @33650305387,MOI,Merci @33650305387
7,20/07/2022 15:43,Cedriques Bampoky: Rebienvenue parmi tes frères,Cedriques Bampoky,Rebienvenue parmi tes frères


In [14]:
ami.drop(["Messages"],axis =1, inplace =True)

In [15]:
ami.head()

,date,auteur,SMS
3,20/07/2022 15:42,MOI,"Bonjour,"
4,20/07/2022 15:42,Je n'ai pas vu Macky 🤣🤣🤣,Waly
5,20/07/2022 15:42,MOI,Il est finalement parti avant 2024 ? 🥲
6,20/07/2022 15:43,MOI,Merci @33650305387
7,20/07/2022 15:43,Cedriques Bampoky,Rebienvenue parmi tes frères


In [16]:
idx1 =ami[ami["SMS"]=="Waly"].index
ami.loc[idx1].head(3)

,date,auteur,SMS
4,20/07/2022 15:42,Je n'ai pas vu Macky 🤣🤣🤣,Waly
38,21/07/2022 12:40,Paix à son âme,Waly
69,23/07/2022 16:57,‎Cedriques Bampoky a ajouté Henry Thiare,Waly


In [17]:
for i in idx1:
    ami.loc[i]["SMS"] = ami.loc[i]["auteur"]

In [18]:
ami.head()

,date,auteur,SMS
3,20/07/2022 15:42,MOI,"Bonjour,"
4,20/07/2022 15:42,Je n'ai pas vu Macky 🤣🤣🤣,Je n'ai pas vu Macky 🤣🤣🤣
5,20/07/2022 15:42,MOI,Il est finalement parti avant 2024 ? 🥲
6,20/07/2022 15:43,MOI,Merci @33650305387
7,20/07/2022 15:43,Cedriques Bampoky,Rebienvenue parmi tes frères


In [19]:
ami.loc[[253,254,255]]["date"] ="30/07/2022 20:45"
ami.loc[[285,286,287,288,289]]["date"] ="31/07/2022 16:38"
ami.loc[[242]]["date"] = "29/07/2022 08:13"

In [20]:
ami.tail()

,date,auteur,SMS
404,15/08/2022 18:58,Nicolas Faye,Merci tonton justin.
405,15/08/2022 18:58,Binne fete a toi de meme,Binne fete a toi de meme
406,15/08/2022 19:28,Simon Issa,Bonne fête à tout le monde
407,15/08/2022 19:29,Simon Issa,Mais Nicodème tu es injoignable dehhhh
408,15/08/2022 19:56,Achille Diop,Ou son les disciplines de battus


In [21]:
for i in ami.index:
    if len(ami.loc[i]["auteur"].split()) >2:
        ami.loc[i]["auteur"] = ami.loc[i-1]["auteur"]
        

In [22]:
ami.head()

,date,auteur,SMS
3,20/07/2022 15:42,MOI,"Bonjour,"
4,20/07/2022 15:42,MOI,Je n'ai pas vu Macky 🤣🤣🤣
5,20/07/2022 15:42,MOI,Il est finalement parti avant 2024 ? 🥲
6,20/07/2022 15:43,MOI,Merci @33650305387
7,20/07/2022 15:43,Cedriques Bampoky,Rebienvenue parmi tes frères


In [23]:
ami["auteur"].value_counts()

 MOI                  69
 Simon Issa           59
 Theo Ndong           50
 Nicolas Faye         48
 Justin Sène          42
 Achille Diop         38
 Cedriques Bampoky    33
 Pape Faye            31
 Ounithe              17
 Semou Diouf           7
 Seck Diouf            6
end électoral.         3
Diadieuf               1
L’autre disait         1
IPAN                   1
Name: auteur, dtype: int64

In [24]:
col_sup = [242,253,254,255,285,286,287,288,289,290,291,292,293,294,295,296]
ami = ami.drop(col_sup,axis =0)

In [25]:
ami.reset_index(drop = True,inplace = True)
ami.head()

,date,auteur,SMS
0,20/07/2022 15:42,MOI,"Bonjour,"
1,20/07/2022 15:42,MOI,Je n'ai pas vu Macky 🤣🤣🤣
2,20/07/2022 15:42,MOI,Il est finalement parti avant 2024 ? 🥲
3,20/07/2022 15:43,MOI,Merci @33650305387
4,20/07/2022 15:43,Cedriques Bampoky,Rebienvenue parmi tes frères


In [26]:
ami["date"] = pd.to_datetime(ami["date"])
ami = ami.set_index("date")

In [27]:
ami

,auteur,SMS
date,,
2022-07-20 15:42:00,MOI,"Bonjour,"
2022-07-20 15:42:00,MOI,Je n'ai pas vu Macky 🤣🤣🤣
2022-07-20 15:42:00,MOI,Il est finalement parti avant 2024 ? 🥲
2022-07-20 15:43:00,MOI,Merci @33650305387
2022-07-20 15:43:00,Cedriques Bampoky,Rebienvenue parmi tes frères
...,...,...
2022-08-15 18:58:00,Nicolas Faye,Merci tonton justin.
2022-08-15 18:58:00,Nicolas Faye,Binne fete a toi de meme
2022-08-15 19:28:00,Simon Issa,Bonne fête à tout le monde


In [28]:
ami["auteur"].value_counts()

 MOI                  69
 Theo Ndong           50
 Simon Issa           49
 Nicolas Faye         48
 Justin Sène          42
 Achille Diop         38
 Cedriques Bampoky    33
 Pape Faye            30
 Ounithe              17
 Semou Diouf           7
 Seck Diouf            6
Diadieuf               1
Name: auteur, dtype: int64

In [29]:
ami[ami["auteur"]=="Diadieuf"].index

DatetimeIndex(['2022-07-24 11:03:00'], dtype='datetime64[ns]', name='date', freq=None)

In [30]:
ami["auteur"].unique()

array([' MOI', ' Cedriques Bampoky', ' Justin Sène', ' Theo Ndong',
       ' Ounithe', ' Simon Issa', ' Nicolas Faye', ' Pape Faye',
       ' Semou Diouf', ' Achille Diop', 'Diadieuf', ' Seck Diouf'],
      dtype=object)

In [31]:
ami[ami["auteur"]==' Cedriques Bampoky']

,auteur,SMS
date,,
2022-07-20 15:43:00,Cedriques Bampoky,Rebienvenue parmi tes frères
2022-07-20 15:44:00,Cedriques Bampoky,Pas de débat politique sinon di nala door pét...
2022-07-20 16:05:00,Cedriques Bampoky,Dinala dieul ci souf✌🏿👍🏿
2022-07-20 20:00:00,Cedriques Bampoky,Dama titt bimako guissé
2022-07-20 20:10:00,Cedriques Bampoky,😂😂😂😂
2022-07-21 13:13:00,Cedriques Bampoky,Ayo😭😭 triste nouvelle que son âme repose en p...
2022-07-23 16:57:00,Cedriques Bampoky,Je lui ai envoyé une invitation
2022-07-23 22:52:00,Cedriques Bampoky,Merci Dr ✌🏿👍🏿
2022-07-23 22:52:00,Cedriques Bampoky,Limay raté évènement yi méti nama dh weuhhhh😭😭😭😭


In [32]:
ami[ami["auteur"]==' Semou Diouf']

,auteur,SMS
date,,
2022-07-21 14:55:00,Semou Diouf,Paix à son âme.
2022-07-24 15:05:00,Semou Diouf,Mes félicitations mon frère.
2022-07-24 15:05:00,Semou Diouf,Je suis sur le terrain avec le Loup Seck Diouf
2022-11-08 23:46:00,Semou Diouf,Félicitations bro.
2022-11-08 23:46:00,Semou Diouf,Tu maîtrise ton sujet.
2022-11-08 23:48:00,Semou Diouf,Félicitations aussi à Justin pour son interve...
2022-11-08 23:48:00,Semou Diouf,Bravo Camarade de Classe💪


In [33]:
ami[ami["auteur"]==' Achille Diop']

,auteur,SMS
date,,
2022-07-24 10:10:00,Achille Diop,Bonjour ms frère un big merci à vs tous ceux ...
2022-07-24 10:30:00,Achille Diop,Bro j ss routs
2022-07-24 10:31:00,Achille Diop,J compren la pluie à bcp jouer du fai certain...
2022-07-24 10:50:00,Achille Diop,J en doute pa
2022-07-24 10:51:00,Achille Diop,Lol
2022-07-24 10:51:00,Achille Diop,Venai pti dej au lit
2022-07-24 11:39:00,Achille Diop,Bane affer waly
2022-07-24 11:40:00,Achille Diop,Ah kone li yeup diotenafi am sama ginaw
2022-07-24 11:41:00,Achille Diop,Kone théo yow ya deff li


In [34]:
ami[ami["auteur"]==' Seck Diouf']

,auteur,SMS
date,,
2022-07-24 14:50:00,Seck Diouf,Bjr
2022-07-24 14:50:00,Seck Diouf,Mes félicitations achil
2022-07-24 14:50:00,Seck Diouf,Nek thi campagne bi rek
2022-07-30 20:49:00,Seck Diouf,Lamb nak
2022-11-08 11:04:00,Seck Diouf,Djadjeuf waye docteur simon
2022-11-08 11:04:00,Seck Diouf,Nouma néne


In [35]:
ami[ami["auteur"]==' Ounithe']

,auteur,SMS
date,,
2022-07-21 09:02:00,Ounithe,Merci imam Pape.
2022-07-21 09:19:00,Ounithe,Cette infirmière a des seins tueurs.
2022-07-24 00:45:00,Ounithe,Djadjeuf samaa gayii. Sañsé ngeen deh. Machala 🙏
2022-07-24 14:15:00,Ounithe,Le cas devient communautaire alors. 🤔🤔
2022-07-24 14:16:00,Ounithe,Métisse moom ça me va bien
2022-07-24 14:21:00,Ounithe,C'est pas une question de mandats jumeau 🤣 C'...
2022-07-24 14:23:00,Ounithe,TONTON Achille yaw amo morom
2022-07-24 14:42:00,Ounithe,Élever 👌👌voilà le bon terme. Merci jumeau. Ma...
2022-07-24 21:07:00,Ounithe,D'autant plus que c'est un cas litigieux. ✍️


In [36]:
ami[ami["auteur"]==' Theo Ndong']

,auteur,SMS
date,,
2022-07-20 20:03:00,Theo Ndong,Saints seins 🥰🥰🥰
2022-07-21 15:14:00,Theo Ndong,Salut la famille !
2022-07-21 15:14:00,Theo Ndong,Union de prières spéciale pour le repos de so...
2022-07-21 22:17:00,Theo Ndong,Veup Bro
2022-07-21 22:18:00,Theo Ndong,"D'hab on garde jusqu'au jour, régler des impr..."
2022-07-21 22:27:00,Theo Ndong,Individuellement ?
2022-07-21 22:57:00,Theo Ndong,"Ça a dû t'échapper,on avait retenu 20.000f ce..."
2022-07-21 22:57:00,Theo Ndong,Super !!
2022-07-22 00:11:00,Theo Ndong,"SIF pour les cartes comme d'hab, venir avec l..."


In [37]:
ami[ami["auteur"]==' Justin Sène']

,auteur,SMS
date,,
2022-07-20 19:26:00,Justin Sène,🤣
2022-07-21 10:10:00,Justin Sène,Wa Simon khana c'est toi🤣🤣🤣
2022-07-21 10:10:00,Justin Sène,Ki mome yako inspiré
2022-07-21 12:40:00,Justin Sène,Triste nouvelle boy
2022-07-21 12:40:00,Justin Sène,Paix à son âme
2022-07-22 00:00:00,Justin Sène,👍🏿
2022-07-23 20:38:00,Justin Sène,Motna
2022-07-24 11:01:00,Justin Sène,Bjr Waly. Achille tu es matinal hein.
2022-07-24 11:01:00,Justin Sène,J'espère que yangui récupéré légui.


In [38]:
ami[ami["auteur"]== ' Pape Faye']

,auteur,SMS
date,,
2022-07-21 12:46:00,Pape Faye,Inna illah wa inna ilayhi raaji’uun.
2022-07-21 22:17:00,Pape Faye,Dom dab bro
2022-07-22 07:48:00,Pape Faye,Jummah Mubarak
2022-07-23 18:46:00,Pape Faye,Diadieuf
2022-07-23 20:37:00,Pape Faye,@221774071233 fou signature yi tolou
2022-07-23 20:40:00,Pape Faye,Ok
2022-07-23 23:01:00,Pape Faye,Massa way bro
2022-07-24 11:59:00,Pape Faye,Bjr la mifa
2022-07-24 12:13:00,Pape Faye,Yaguena


In [39]:
ami.drop(['2022-07-24 11:03:00'],inplace = True)

In [40]:
ami["auteur"].value_counts()

 MOI                  68
 Theo Ndong           50
 Simon Issa           49
 Nicolas Faye         48
 Justin Sène          42
 Achille Diop         38
 Cedriques Bampoky    33
 Pape Faye            30
 Ounithe              17
 Semou Diouf           7
 Seck Diouf            6
Name: auteur, dtype: int64

In [41]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
def sentimentScore(sentences):
    sia = SentimentIntensityAnalyzer()
    Results=pd.DataFrame()
    for sentence in sentences:
        result = sia.polarity_scores(sentence)
        result = pd.DataFrame([result])
        Results=pd.concat([Results,result],axis = 0,ignore_index=True)
    return Results

In [42]:
Vad_result=sentimentScore(list(ami["SMS"].values))
Vad_result.head()

,neg,neu,pos,compound
0,0.0,1.0,0.0,0.0
1,0.0,1.0,0.0,0.0
2,0.0,1.0,0.0,0.0
3,0.0,1.0,0.0,0.0
4,0.0,1.0,0.0,0.0


In [43]:
y_Vadpred=[]
for i in range(Vad_result.shape[0]):
    if Vad_result.loc[i]["pos"] > Vad_result.loc[i]["neg"]:
         y_Vadpred.append(4)
    elif Vad_result.loc[i]["pos"] < Vad_result.loc[i]["neg"]:
        y_Vadpred.append(0)
    else:
        y_Vadpred.append(2)  

In [44]:
y_pred = pd.Series(y_Vadpred)

#### Nombre de messages positifs

In [45]:
(y_pred>2).sum()

22

### Nombre de messages négatifs

In [46]:
(y_pred<2).sum()

12

###  Nombre de messages neutres

In [47]:
(y_pred==2).sum()

354